# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import pprint
import time
import os

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Loan CSV file generated from WeatherPy Folder
weather_data_csv = "../WeatherPy/output_data/output_weather_data_file.csv"
weather_data_df = pd.read_csv(weather_data_csv)
weather_data_df.head()

,Unnamed: 0,city name,country,city id,latitude_deg,longitude_deg,temp_max_F,humidity_%,cloudiness_%,winds speed_mph
0,0,Yellowknife,CA,6185377,62.46,-114.35,1.99,84,75,3.36
1,1,Cabo San Lucas,MX,3985710,22.89,-109.91,89.01,58,5,3.36
2,2,Ushuaia,AR,3833367,-54.80,-68.30,64.40,36,0,5.82
3,3,Hilo,US,5855927,19.73,-155.09,80.60,74,90,5.82
4,4,Marti,CU,3547519,22.95,-80.92,77.00,94,75,17.22


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Locations
locations = weather_data_df[["latitude_deg", "longitude_deg"]]
#locations
type(locations)
## Convert humidity to float and store.  we will use it for wt.
humidity = weather_data_df["humidity_%"].astype(float)
type(humidity)

pandas.core.series.Series

In [5]:
#Plot heat map
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2)
# Add layer
fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Perfect vacation city = max temp = 65F-85F, humidity = 45% to 60%, cloudiness <30 

city_above_65f = weather_data_df[weather_data_df["temp_max_F"] > 65]
city_65f_to_85f = city_above_65f[city_above_65f["temp_max_F"] < 88]

city_above_45humid =city_65f_to_85f [city_65f_to_85f ["humidity_%"]>45] 
city_45humid_to_60humid = city_above_45humid[city_above_45humid["humidity_%"]<60]

perfect_city = city_45humid_to_60humid[city_45humid_to_60humid["cloudiness_%"] < 30]

#perfect_city
#del perfect_city['Unnamed: 0']
indexed_perfect_city = perfect_city.reset_index()
indexed_perfect_city = indexed_perfect_city.drop(['index', 'Unnamed: 0'], axis=1)
perfect_cities = indexed_perfect_city
perfect_cities

#setup additional column
perfect_cities["hotels"]=""

perfect_cities.head()

,city name,country,city id,latitude_deg,longitude_deg,temp_max_F,humidity_%,cloudiness_%,winds speed_mph,hotels
0,Pacific Grove,US,5380437,36.62,-121.92,66.99,59,1,3.36,
1,Lompoc,US,5367788,34.64,-120.46,75.20,49,1,6.93,
2,Touros,BR,3386213,-5.20,-35.46,86.00,58,20,17.22,
3,Utinga,BR,3392998,-5.82,-35.40,86.00,58,20,17.22,
4,Mar Del Plata,AR,3430863,-38.00,-57.56,69.80,49,0,19.46,


In [7]:
##create perfect city dataframe with location
perfect_locations = perfect_city[["latitude_deg", "longitude_deg"]]

##create a series of perfect humidity for perfect city.  will use it for wt
perfect_humidity = perfect_city["humidity_%"].astype(float)

# Plot Heatmap
perfect_fig = gmaps.figure()

# Create heat layer
perfect_heat_layer = gmaps.heatmap_layer(perfect_locations, weights=perfect_humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2.5)

# Add layer
perfect_fig.add_layer(perfect_heat_layer)

# Display figure
perfect_fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
hotels = []
for city in range(len(perfect_cities["city name"])):

    #target search
    lat = perfect_city.loc[:,"latitude_deg"]
    long = perfect_city.loc[:,"longitude_deg"]  
    target_lat_long = f"{lat},{long}"
    target_type = "lodging"
    radius = 5000
    
    params = {"location": target_lat_long, 
        "type": target_type,
        "radius": radius,
        "key": g_key}

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels_request = requests.get(base_url,params=params)
    hotel_response = hotels_request.json()
    print(hotel_response)
    #try:
        #hotels.append(hotel_response["results"][0]["name"])
    #except:
        #hotels.append("Nearest hotel not found")
        

        
        
# Dataframe with nearest hotel
#perfect_cities["Nearest Hotel"] = hotels
#hotels


{'error_message': 'You have exceeded your daily request quota for this API. If you did not set a custom daily request quota, verify your project has an active billing account: http://g.co/dev/maps-no-account', 'html_attributions': [], 'results': [], 'status': 'OVER_QUERY_LIMIT'}
{'error_message': 'You have exceeded your daily request quota for this API. If you did not set a custom daily request quota, verify your project has an active billing account: http://g.co/dev/maps-no-account', 'html_attributions': [], 'results': [], 'status': 'OVER_QUERY_LIMIT'}
{'error_message': 'You have exceeded your daily request quota for this API. If you did not set a custom daily request quota, verify your project has an active billing account: http://g.co/dev/maps-no-account', 'html_attributions': [], 'results': [], 'status': 'OVER_QUERY_LIMIT'}
{'error_message': 'You have exceeded your daily request quota for this API. If you did not set a custom daily request quota, verify your project has an active b

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
